https://tproger.ru/translations/markov-chains/

# Цепи Маркова для автоматической генерации тектов

## Структура данных Dictogram

Dictogram (dict — встроенный тип данных словарь в Python) будет отображать зависимость между звеньями и их частотой появления в тексте, то есть их распределение. Но при этом она будет обладать нужным нам свойством словаря — время выполнения программы не будет зависеть от объема входных данных, а это значит, мы создаем эффективный алгоритм.

In [1]:
import random

class Dictogram(dict):
    def __init__(self, iterable=None):
        # Инициализируем наше распределение как новый объект класса, 
        # добавляем имеющиеся элементы
        super(Dictogram, self).__init__()
        self.types = 0  # число уникальных ключей в распределении
        self.tokens = 0  # общее количество всех слов в распределении
        if iterable:
            self.update(iterable)

    def update(self, iterable):
        # Обновляем распределение элементами из имеющегося 
        # итерируемого набора данных
        for item in iterable:
            if item in self:
                self[item] += 1
                self.tokens += 1
            else:
                self[item] = 1
                self.types += 1
                self.tokens += 1

    def count(self, item):
        # Возвращаем значение счетчика элемента, или 0
        if item in self:
            return self[item]
        return 0

    def return_random_word(self):
        random_key = random.sample(self, 1)
        # Другой способ:
        # random.choice(histogram.keys())
        return random_key[0]

    def return_weighted_random_word(self):
        # Сгенерировать псевдослучайное число между 0 и (n-1),
        # где n - общее число слов
        random_int = random.randint(0, self.tokens-1)
        index = 0
#         list_of_keys = 
        # вывести 'случайный индекс:', random_int
        for key in self.keys():
            index += self[key]
            # вывести индекс
            if(index > random_int):
                # вывести list_of_keys[i]
                return key

В конструктор структуре Dictogram можно передать любой объект, по которому можно итерироваться. Элементами этого объекта будут слова для инициализации Dictogram, например, все слова из какой-нибудь книги. В данном случае мы ведем подсчет элементов, чтобы для обращения к какому-либо из них не нужно было пробегать каждый раз по всему набору данных.

Мы также сделали две функции для возврата случайного слова. Одна функция выбирает случайный ключ в словаре, а другая, принимая во внимание число появлений каждого слова в тексте, возвращает нужное нам слово.

## Структура цепи Маркова

In [2]:
def make_markov_model(data):
    markov_model = dict()

    for i in range(0, len(data)-1):
        if data[i] in markov_model:
            # Просто присоединяем к уже существующему распределению
            markov_model[data[i]].update([data[i+1]])
        else:
            markov_model[data[i]] = Dictogram([data[i+1]])
    return markov_model

В реализации выше у нас есть словарь, который хранит окна в качестве ключа в паре «(ключ, значение)» и распределения в качестве значений в этой паре.

## Cтруктура цепи Маркова N-го порядка

In [3]:
def make_higher_order_markov_model(order, data):
    markov_model = dict()

    for i in range(0, len(data)-order):
        # Создаем окно
        window = tuple(data[i: i+order])
        # Добавляем в словарь
        if window in markov_model:
            # Присоединяем к уже существующему распределению
            markov_model[window].update([data[i+order]])
        else:
            markov_model[window] = Dictogram([data[i+order]])
    return markov_model

## Парсинг модели

Отлично, мы реализовали словарь. Но как теперь совершить генерацию контента, основываясь на текущем состоянии и шаге к следующему состоянию? Пройдемся по нашей модели:

In [4]:
from collections import deque
import re


def generate_random_start(model):
    # Чтобы сгенерировать любое начальное слово, раскомментируйте строку:
    # return random.choice(model.keys())

    # Чтобы сгенерировать "правильное" начальное слово, используйте код ниже:
    # Правильные начальные слова - это те, что являлись началом предложений в корпусе
    if 'END' in model:
        seed_word = 'END'
        while seed_word == 'END':
            seed_word = model['END'].return_weighted_random_word()
        return seed_word
    return random.choice(list(model.keys()))


def generate_random_sentence(length, markov_model):
    current_word = generate_random_start(markov_model)
    sentence = [current_word]
    for i in range(0, length):
        current_dictogram = markov_model[current_word]
        random_weighted_word = current_dictogram.return_weighted_random_word()
        current_word = random_weighted_word
        sentence.append(current_word)
    sentence[0] = sentence[0].capitalize()
    return ' '.join(sentence) + '.'
    return sentence

Лучше генерировать «правильные» начальные слова – те, которые и в исходном тексте стояли в начале предложения. Для этого мы в словаре находим все ключи «END» и выбираем слово, следующее за одним из них. После генерации начального слова мы ищем, какое слово может идти дальше, обращаясь к тому же словарю, и выбираем нужное на основании комбинации вероятности и случайности. Продолжаем это делать, пока предложение не достигнет установленной нами длины, и в конце возвращаем его. 

## Примеры

In [8]:
text = open('nietzsche.txt').read()
text = text.replace("»", " END").replace("«", "").replace("\n", " ").replace(".", " END").replace(",", "")
frags = [f.lower() if f != 'END' else f for f in text.split(" ") if f]
print(len(frags))

101895


### Для 1-го порядка

In [9]:
model = make_markov_model(frags)

In [10]:
generate_random_sentence(100, model).replace(" END", ".")

'It undermines and we now pleases thee? what have grown utilitarian qualities which confers upon the place thousands of a time it is distrust which on the european of repugnance in mozart--how happy and the moments in the church has hitherto has expressed it: "what i saying! to conjure back to the countries where he must however is comprised in art and love of the non-natural supernatural because he will be doubly distrustful here a surplus of humanity and disciplining influence--destructive as a man\'s faith in the "evil principle to his strength--the greatest liar--and the philosophy at the structure of mind.'

### Доделано: Для N-го порядка

In [10]:
model = make_higher_order_markov_model(4, frags)

In [12]:
def triplet(model, w=''): # функция генерирует случайный первый триплет, начинающийся на END
    s = []
    if w == 'END':
        for i in model:
            if i[0] == 'END':
                s.append(i)
        n = random.randrange(len(s))
        return(s[n])
    else:
        n = random.randrange(len(list(model.keys())))
        return(list(model.keys())[n])

In [13]:
def generate(model, length, n=3): # n - это длина "окна", должна совпадать с аналогичным параметром модели
    current = triplet(model, w='END')
    sentence = [i for i in current]
    for i in range(3, length+1):
        #print(current)
        if tuple(current) in model: 
            current_dictogram = model[tuple(current)]
            random_weighted_word = current_dictogram.return_weighted_random_word()
            #print(random_weighted_word)
            sentence.append(random_weighted_word)
            #print(sentence)
            current = sentence[i-n+1:i+n]
            #print(current)
        else:
            #print(current)
            random_word = triplet(model)
            sentence.extend([i for i in random_word])
            #print(sentence)
            current = sentence[i-n+1:i+n]
            
    sentence = sentence[1:]
    sentence[0] = sentence[0].capitalize()
    for i in range (len(sentence)-1):
        if sentence[i] == 'END':
            sentence[i+1] = sentence[i+1].capitalize()
    res = ' '.join(sentence) + '.'
    res = re.sub(' END', '.',  res)
    return res

t = generate(model, 300, 4)
t

'The phenomena remain unexplained and processes have really employment of a unity wanton surrender to the year\'s end alone with indignation and enthusiasm so. 144 it stands in the general happiness entertain the idea of. 134 if now the metaphysical sense: but the restricted sense of [of duration or measurement] know them are permitted of heredity or natural to an end just to whom everything in owes to metaphysics. 105. The pia expediency mixed with stupidity truths as if by advanced by a distinguished impulses of his nature the logical from the. The degrees of which seeks to rise even justify themselves by another a place of by the way that supposing that every philosophy same light in which despair in particular does wish to give them magic including therefore the casts of fancy there themselves and push the in no previous age of the loathsome and or semi-idleness is necessary causes philosophers to be ourselves obliged to give there may not be of everything of the say even that the 

# Библиотека Markovify 

In [2]:
import markovify

# Get raw text as string.
with open('nietzsche.txt') as f:
    text = f.read()

text_model = markovify.Text(text)

Yet not in Germany.
He honours whatever he finds veiled or protected by any means as ill as the true explanation and served as such.
It seems, therefore, that however little we may always remain undetermined: but there is no life; the struggle for life.
Indeed the whole Romanticism and sublimity of moral feelings: and rightly.
The individual can, in addition to this probably unpleasant conclusion, now that the imagination were the first of the future--as certainly also they will suffer inwardly from a thing, is to tempt us to accept bad grounds as good.
Here one must think profoundly to the sense of the recurring and more dangerous sense?
The custom is exceedingly burdensome it is a delusion.
In relation to the more confused that origin is.


In [3]:
for i in range(5):
    print(text_model.make_sentence())

Unfortunately, however, it is regarded as necessary, as a power--how COULD you live in a threatening manner what woman first and foremost--old Kant.
Oh, ye demons, can ye not that it will flee from them!
In brief, the system of utilitarianism, one may say that the animal has become strong enough, then, to cause the world apparently most indifferent and unprejudiced, and has fought and died for his disciplining and educating work, just as they are not done to the following:--I have never yet met a German translation of Petronius, who, more than a disadvantage to mankind only after long struggle and wavering!
Not to cleave to a science, though it were the only means of magic between man and woman never cease marvelling and laughing; does it matter!
One can never be wholly clarified and made its first entrance into Germany in the act.


In [4]:
for i in range(3):
    print(text_model.make_short_sentence(140))

Why is the symptom.
People had been blunted by the realm of stars.
The proof of the world.
